Fuente de los datos: https://www.mapa.gob.es/es/estadistica/temas/publicaciones/informe-semanal-coyuntura/2020.aspx

*Los datos consistían en un archivo excel por cada semana del año 2020 que contenían los precios medios nacionales de varios productos. Los de frutas y hortalizas estaban en la página 5. Por esa razón los primeros pasos (que están comentados porque ya se han hecho y así facilitar la ejecución) permiten extraer la página 5 y convertirla en un archivo csv con el nombre df_(semana del año). Ese es el punto de partida del código.*

In [1]:
# Se cargan las librerías
import os
import pandas as pd

In [2]:
# Se crea una lista con todos los archivos de excel de los datos de precios por producto y por semana del año 2020
# files = os.listdir()
# frames = []
# files_xlsx = [f for f in files if f[-4:] == 'xlsx']

In [3]:
# Se crea un dataframe que lea la página 5 (donde están los precios de furtas y hortalizas) y se borran las columnas sobrantes
# Se renombran las columnas que nos interesan (nombre de producto y precio en euros/kg de ese producto)
# Se elimina la cabecera y el final de la pestaña del excel
# Se crea un csv con el nombre 'df_(número de la semana del año)'
# for file in files_xlsx:
#     df = pd.read_excel(file,'Pág. 5')
#     df.drop(columns = ['Unnamed: 0','Unnamed: 1','Unnamed: 3','Unnamed: 5','Unnamed: 6'],inplace = True)
#     df.rename(columns =  {'Unnamed: 2':'PRODUCTOS','Unnamed: 4': 'Semana'+ str(file[27:29])}, inplace = True)
#     df = df.iloc[2:36,:]
#     df.to_csv('df_' + file[27:29] + '.csv',encoding='utf-8-sig')
    

*EJECUTAR DESDE AQUÍ PORQUE LOS PASOS ANTERIORES PREVIOS YA ESTÁN HECHOS (YA SE HAN CREADO LOS ARCHIVOS CSV)*

In [4]:
# Se crea una lista llamada files_csv (que tiene el nombre de todos los archivos csv del directorio)
# Se itera por cada uno de los archivos csv para eliminar las columnas sobrantes y pasar los datos numéricos a float
# También se renombran columnas que en algunos archivos tienen un nombre ligeramente distinto
# Se pone la columna PRODUCTOS como índice y se juntan todos los dataframes en frames (lista)
frames = []
files = os.listdir()
files_csv = [f for f in files if f[-3:] == 'csv']
for file in files_csv:
    df = pd.read_csv(file, encoding = 'utf-8-sig')
    df.drop(columns = ['Unnamed: 0'], inplace = True)
    df.iloc[:,1] = pd.to_numeric(df.iloc[:,1], errors='coerce')
    df = df.iloc[3:36,:]
    df.dropna(inplace = True)
    df['PRODUCTOS'] = df['PRODUCTOS'].replace('Escarola (€/100 ud)','Escarola (€/100 kg)')
    df['PRODUCTOS'] = df['PRODUCTOS'].replace('Judía verde tipo plana (€/100 kg)*','Judía verde tipo plana (€/100 kg)',)
    df['PRODUCTOS'] = df['PRODUCTOS'].replace('Col-repollo hoja lisa (€/100 kg)','Col-repollo (€/100kg)')
    df.set_index('PRODUCTOS', inplace = True)
    frames.append(df)

In [5]:
 # Se crea el dataframe df_precios (vacío donde hacer un merge de los datasets anteriores)
# Se realiza un bucle para mezclar todos los datasets con una uníon outer (para no perder productos ya que no todos tienen los mismos)
df_precios = pd.DataFrame()
for i in range(len(frames)-1):
    if i == 0:
        df_precios = frames[i].merge(frames[i+1], left_on = ['PRODUCTOS'],right_on = ['PRODUCTOS'], how = 'outer')    
    else:
        df_precios = df_precios.merge(frames[i+1], left_on = ['PRODUCTOS'],right_on = ['PRODUCTOS'], how = 'outer')

In [6]:
# Se resetea el índice para poder acceder más comodamente por las posiciones
df_precios.reset_index(inplace=True)

In [7]:
# Se sustituyen los valores NaN por un 0 
# Hay columnas, por ejemplo las naranjas, que tienen diferente nombre pero se pueden sumar los valores para rellenar los meses
# en los que en algunas categorías no hay datos y en otras sí
df_precios.fillna(0,inplace=True)
df_precios.iloc[2,1:53] = df_precios.iloc[2,1:53] + df_precios.iloc[40,1:53] # naranjas
df_precios.iloc[13,1:53] = df_precios.iloc[2,1:53] + df_precios.iloc[45,1:53] # coliflor
df_precios.iloc[5,1:53] = df_precios.iloc[5,1:53] + df_precios.iloc[42,1:53] # pera blanquilla

In [8]:
# Se saca una lista con los nombres de los productos (ya que algunos apenas tienen valores y se van a eliminar)
df_precios['PRODUCTOS'].unique().tolist()

['Clementina  (€/100 kg)',
 'Limón  (€/100 kg)',
 'Naranja  (€/100 kg)',
 'Aguacate (€/100 kg)',
 'Manzana Golden (€/100 kg)',
 'Pera Blanquilla  (€/100kg)',
 'Plátano (€/100 kg)',
 'Acelga (€/100kg)',
 'Alcachofa (€/100kg)',
 'Berenjena (€/100 kg)',
 'Calabacín (€/100 kg)',
 'Cebolla (€/100 kg)',
 'Champiñón (€/100kg)',
 'Coliflor (€/100kg)',
 'Col-repollo (€/100kg)',
 'Escarola (€/100 kg)',
 'Fresa (€/100 kg)',
 'Haba verde (€/100 kg)',
 'Judía verde tipo plana (€/100 kg)',
 'Lechuga Romana (€/100 ud)',
 'Pepino (€/100 kg)',
 'Pimiento verde tipo italiano (€/100 kg)',
 'Puerro (€/100 kg)',
 'Tomate liso (€/100 kg)',
 'Zanahoria (€/100 kg) ',
 'Patata (€/100 kg) ',
 'Espinaca (€/100 kg)',
 'Espárrago (€/100kg)',
 'Níspero (€/100 kg)',
 'Albaricoque (€/100 kg)',
 'Melocotón (€/100 kg)',
 'Cereza (€/100 kg)',
 'Nectarina (€/100 kg)',
 'Ajo (€/100kg)',
 'Naranja - Grupo Blancas (€/100 kg)',
 'Sandía (€/100 kg)',
 'Ciruela (€/100 kg)',
 'Higo / breva (€/100 kg)',
 'Melón Piel de Sapo (€/1

In [9]:
# Se crea una nueva lista con los valores eliminados
lista_precios = ['Clementina  (€/100 kg)','Limón  (€/100 kg)','Naranja  (€/100 kg)','Aguacate (€/100 kg)','Manzana Golden (€/100 kg)',
 'Pera Blanquilla  (€/100kg)','Plátano (€/100 kg)','Acelga (€/100kg)','Alcachofa (€/100kg)','Berenjena (€/100 kg)','Calabacín (€/100 kg)',
 'Cebolla (€/100 kg)','Champiñón (€/100kg)','Coliflor (€/100kg)','Col-repollo (€/100kg)','Escarola (€/100 kg)','Fresa (€/100 kg)',
 'Haba verde (€/100 kg)','Judía verde tipo plana (€/100 kg)','Lechuga Romana (€/100 ud)','Pepino (€/100 kg)','Pimiento verde tipo italiano (€/100 kg)',
 'Puerro (€/100 kg)','Tomate liso (€/100 kg)','Zanahoria (€/100 kg) ','Patata (€/100 kg) ','Espinaca (€/100 kg)',
 'Melocotón (€/100 kg)','Ajo (€/100kg)','Uva de mesa (€/100 kg)','Satsuma (€/100 kg)','Caqui (€/100 kg)','Pera Conferencia (€/100 kg)',
 'Granada (€/100 kg)','Brócoli (€/100 kg)','Mandarina (€/100 kg)']

In [10]:
# Se realiza una máscara en la que se incluyen únicamente los productos que están en esa lista
df_precios = df_precios[df_precios['PRODUCTOS'].isin(lista_precios)]

In [11]:
df_precios

,PRODUCTOS,Semana01,Semana02,Semana03,Semana04,Semana05,Semana06,Semana07,Semana08,Semana09,...,Semana44,Semana45,Semana46,Semana47,Semana48,Semana49,Semana50,Semana51,Semana52,Semana53
0,Clementina (€/100 kg),35.849414,38.403939,40.529336,40.865100,39.030314,45.760227,61.500000,61.500000,0.000000,...,33.391775,31.235710,27.313045,24.802632,24.246112,22.734427,21.527706,20.087304,19.201966,19.308570
1,Limón (€/100 kg),41.223680,40.044239,41.104289,41.312064,41.312064,40.059215,39.742342,40.359104,39.509048,...,34.132194,33.893837,34.641093,29.619178,26.072605,25.000000,23.877902,22.341573,21.024720,19.061049
2,Naranja (€/100 kg),24.302470,24.622900,25.676555,25.765075,26.980960,30.330797,30.691755,28.805540,28.208983,...,24.996707,25.134175,24.377224,24.483059,25.197381,25.811975,24.925747,24.160550,23.355232,0.000000
3,Aguacate (€/100 kg),215.701545,217.713980,216.940353,219.539852,239.447784,242.690032,239.606617,231.677695,228.296649,...,199.567485,186.612059,182.138159,171.212002,172.874754,175.085125,179.336638,178.310773,188.887058,190.733132
4,Manzana Golden (€/100 kg),39.782394,37.544579,37.814990,37.675824,39.183504,38.862603,38.570445,38.696128,38.674079,...,54.493025,53.779544,54.967096,58.532678,56.682730,58.140553,56.717355,57.780010,57.780010,57.357873
5,Pera Blanquilla (€/100kg),62.006281,62.006281,66.017654,66.017654,66.017654,65.844007,66.844450,66.844450,67.344672,...,53.747249,53.099525,53.099525,53.099525,53.099525,50.797042,50.797042,50.797042,50.797042,0.000000
6,Plátano (€/100 kg),33.570000,28.710000,28.760000,28.530000,28.950000,19.600000,17.400000,17.180000,15.850000,...,97.930000,98.030000,88.490000,81.020000,51.370000,31.520000,32.390000,34.400000,32.080000,31.790000
7,Acelga (€/100kg),48.841239,49.572521,58.137988,65.096168,65.751778,61.353963,60.841465,52.513032,53.023980,...,61.480818,61.004222,58.035829,56.024661,54.189463,57.097923,55.344874,56.479912,56.169407,56.579699
8,Alcachofa (€/100kg),93.870616,102.744931,131.455915,131.472573,98.412841,77.361116,67.008707,58.213732,59.650802,...,150.000000,168.520363,163.204237,152.699346,141.576173,152.947212,160.795578,148.671063,117.830295,116.647850
9,Berenjena (€/100 kg),35.607733,86.659769,116.384290,119.880988,112.768573,69.159208,35.871039,29.664438,24.810168,...,33.709034,27.968167,23.596773,18.914680,33.907228,44.385717,79.655026,64.830712,59.662467,67.269880


In [12]:
# Se guarda el dataframe de Precios Nacionales
#df_precios.to_csv("Precios_nacionales.csv",index = False,encoding = 'utf-8-sig')

*Los precios medios iguales a 0 hacen referencia a la falta de datos para esa semana y productos. En muchos casos como conscuencia de la estacionalidad.*